In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import json
import re

class CET4Downloader:
    @staticmethod
    def reverse(data):
        # 解析JSON数据
        parsed_data = json.loads(data[0])
        # 提取f1和f2的元素并反向连接
        f1_elements = parsed_data['f1']
        f2_elements = parsed_data['f2']
        # 反向连接f1和f2的元素
        reversed_string = ''.join(f2_elements[::-1] + f1_elements[::-1])
        return reversed_string

    @staticmethod
    def download_cet4_files():
        url = 'https://zhenti.burningvocabulary.com/cet4'
        head = {'user-agent': 'Bilzzard'}
        html = urlopen(url)
        bs = BeautifulSoup(html, 'html.parser')
        ID = []
        urlList = []
        nameList = []
        pdfUrl = []
        MP3Url = []

        for web_url in bs.find_all('a', {'class': 'link-primary'}):
            W = 'https://zhenti.burningvocabulary.com' + web_url.attrs['href']
            urlList.append(W)
            h = urlopen(Request(W, headers=head))
            bs = BeautifulSoup(h, 'html.parser')
            nameList.append(bs.find('meta', {'name': 'twitter:title'}).attrs['content'])
            response = requests.get(W, headers=head)
            data = re.findall('"fn":(\{"f1":\[.*?\],"f2":\[.*?\]\}),', response.text)
            ID.append(CET4Downloader.reverse(data))
            pdfUrl.append("https://zhenti-res.youshaohua.com/images/read" + web_url.attrs['href']+'/'+CET4Downloader.reverse(data)+'.pdf')
            try:
                MP3Url.append("https://zhenti-res.youshaohua.com/images/read"+web_url.attrs['href']+'/'+CET4Downloader.reverse(data)+'.mp3')
            except IndexError:
                pass

        for i in range(len(ID)):
            pdf_response = requests.get(url=pdfUrl[i], headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"})
            with open("CET4/"+f'{nameList[i]}.pdf','wb')as  f:
                f.write(pdf_response.content)

        for i in range(len(MP3Url)):
            mp3_response = requests.get(url=MP3Url[i], headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"})
            with open("CET4/MP3/"+f'{nameList[i]}.mp3','wb')as  f:
                f.write(mp3_response.content)
cet=CET4Downloader()
cet.download_cet4_files()
